# Storage

> Storage backends for API keys (session and database)

In [ ]:
#| default_exp core.storage

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from typing import Optional, Dict, Any, List
from datetime import datetime, timedelta

from cjm_fasthtml_byok.core.types import (
    APIKey, KeyStorage, StorageBackend, BYOKConfig, UserAPIKey,
    StorageError, KeyNotFoundError
)
from cjm_fasthtml_byok.core.security import KeyEncryptor, get_or_create_app_key

## Session Storage

In [ ]:
#| export
class SessionStorage:
    """
    Session-based storage for API keys.
    Keys are stored in the user's session and expire with the session.
    """
    
    def __init__(
        self,
        config: BYOKConfig  # BYOK configuration object
    ):
        """Initialize session storage with configuration"""
        self.config = config
        self.prefix = config.session_key_prefix
    
    def _get_session_key(
        self,
        provider: str  # Provider name (e.g., 'openai', 'anthropic')
    ) -> str:  # Prefixed session key for the provider
        """Generate session key for a provider"""
        return f"{self.prefix}{provider}"
    
    def store(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        key: APIKey  # API key object to store
    ) -> None:
        """Store an API key in the session"""
        try:
            session_key = self._get_session_key(key.provider)
            request.session[session_key] = key.to_dict()
        except Exception as e:
            raise StorageError(f"Failed to store key in session: {e}")
    
    def retrieve(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        provider: str,  # Provider name to retrieve key for
        user_id: Optional[str] = None  # User ID (unused in session storage)
    ) -> Optional[APIKey]:  # API key object if found and valid, None otherwise
        """Retrieve an API key from the session"""
        session_key = self._get_session_key(provider)
        
        if session_key not in request.session:
            return None
        
        try:
            data = request.session[session_key]
            key = APIKey.from_dict(data)
            
            # Check expiration
            if key.is_expired:
                self.delete(request, provider)
                return None
            
            return key
        except Exception as e:
            raise StorageError(f"Failed to retrieve key from session: {e}")
    
    def delete(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        provider: str,  # Provider name to delete key for
        user_id: Optional[str] = None  # User ID (unused in session storage)
    ) -> None:
        """Delete an API key from the session"""
        session_key = self._get_session_key(provider)
        if session_key in request.session:
            del request.session[session_key]
    
    def list_providers(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        user_id: Optional[str] = None  # User ID (unused in session storage)
    ) -> List[str]:  # List of provider names with stored keys
        """List all providers with stored keys"""
        providers = []
        for key in request.session.keys():
            if key.startswith(self.prefix):
                provider = key[len(self.prefix):]
                providers.append(provider)
        return providers
    
    def clear_all(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        user_id: Optional[str] = None  # User ID (unused in session storage)
    ) -> None:
        """Clear all API keys from the session"""
        keys_to_delete = [k for k in request.session.keys() if k.startswith(self.prefix)]
        for key in keys_to_delete:
            del request.session[key]

## Database Storage

In [ ]:
#| export
class DatabaseStorage:
    """
    Database-backed storage for API keys using SQLAlchemy 2.0+.
    Keys persist across sessions and devices.
    """
    
    def __init__(
        self,
        config: BYOKConfig,  # BYOK configuration object
        db_url: str = "sqlite:///byok_keys.db"  # Database URL (defaults to SQLite)
    ):
        """
        Initialize database storage with SQLAlchemy.
        """
        from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, select, delete
        from sqlalchemy.exc import SQLAlchemyError
        
        self.config = config
        self.engine = create_engine(db_url)
        self.metadata = MetaData()
        
        # Define the table schema
        self.table = Table(
            config.db_table_name,
            self.metadata,
            Column('id', Integer, primary_key=True, autoincrement=True),
            Column('user_id', String, nullable=False),
            Column('provider', String, nullable=False),
            Column('encrypted_value', String, nullable=False),
            Column('created_at', String, nullable=False),
            Column('expires_at', String, nullable=True)
        )
        
        # Create table if it doesn't exist
        self.metadata.create_all(self.engine)
    
    def store(
        self,
        request: Any,  # FastHTML/Starlette request object (unused but kept for interface consistency)
        key: APIKey  # API key object to store in database
    ) -> None:
        """Store an API key in the database"""
        if not key.user_id:
            raise StorageError("user_id is required for database storage")
        
        from sqlalchemy import select, insert, update
        
        try:
            with self.engine.begin() as conn:
                # Check if key already exists
                stmt = select(self.table).where(
                    (self.table.c.user_id == key.user_id) &
                    (self.table.c.provider == key.provider)
                )
                result = conn.execute(stmt).first()
                
                key_data = {
                    'user_id': key.user_id,
                    'provider': key.provider,
                    'encrypted_value': key.encrypted_value.hex(),
                    'created_at': key.created_at.isoformat(),
                    'expires_at': key.expires_at.isoformat() if key.expires_at else None
                }
                
                if result:
                    # Update existing key
                    stmt = update(self.table).where(
                        (self.table.c.user_id == key.user_id) &
                        (self.table.c.provider == key.provider)
                    ).values(**key_data)
                else:
                    # Insert new key
                    stmt = insert(self.table).values(**key_data)
                
                conn.execute(stmt)
                
        except Exception as e:
            raise StorageError(f"Failed to store key in database: {e}")
    
    def retrieve(
        self,
        request: Any,  # FastHTML/Starlette request object (unused but kept for interface consistency)
        provider: str,  # Provider name to retrieve key for
        user_id: Optional[str] = None  # User ID to retrieve key for (required for database)
    ) -> Optional[APIKey]:  # API key object if found and valid, None otherwise
        """Retrieve an API key from the database"""
        if not user_id:
            return None
        
        from sqlalchemy import select
        
        try:
            with self.engine.connect() as conn:
                stmt = select(self.table).where(
                    (self.table.c.user_id == user_id) &
                    (self.table.c.provider == provider)
                )
                result = conn.execute(stmt).first()
                
                if not result:
                    return None
                
                key = APIKey(
                    provider=result.provider,
                    encrypted_value=bytes.fromhex(result.encrypted_value),
                    created_at=datetime.fromisoformat(result.created_at),
                    expires_at=datetime.fromisoformat(result.expires_at) if result.expires_at else None,
                    user_id=result.user_id
                )
                
                # Check expiration
                if key.is_expired:
                    self.delete(request, provider, user_id)
                    return None
                
                return key
                
        except Exception as e:
            raise StorageError(f"Failed to retrieve key from database: {e}")
    
    def delete(
        self,
        request: Any,  # FastHTML/Starlette request object (unused but kept for interface consistency)
        provider: str,  # Provider name to delete key for
        user_id: Optional[str] = None  # User ID to delete key for (required for database)
    ) -> None:
        """Delete an API key from the database"""
        if not user_id:
            return
        
        from sqlalchemy import delete
        
        try:
            with self.engine.begin() as conn:
                stmt = delete(self.table).where(
                    (self.table.c.user_id == user_id) &
                    (self.table.c.provider == provider)
                )
                conn.execute(stmt)
        except Exception as e:
            raise StorageError(f"Failed to delete key from database: {e}")
    
    def list_providers(
        self,
        request: Any,  # FastHTML/Starlette request object (unused but kept for interface consistency)
        user_id: Optional[str] = None  # User ID to list providers for (required for database)
    ) -> List[str]:  # List of provider names with stored keys
        """List all providers with stored keys for a user"""
        if not user_id:
            return []
        
        from sqlalchemy import select, distinct
        
        try:
            with self.engine.connect() as conn:
                stmt = select(distinct(self.table.c.provider)).where(
                    self.table.c.user_id == user_id
                )
                result = conn.execute(stmt)
                return [row[0] for row in result]
        except Exception as e:
            raise StorageError(f"Failed to list providers: {e}")
    
    def clear_all(
        self,
        request: Any,  # FastHTML/Starlette request object (unused but kept for interface consistency)
        user_id: Optional[str] = None  # User ID to clear keys for (required for database)
    ) -> None:
        """Clear all API keys for a user"""
        if not user_id:
            return
        
        from sqlalchemy import delete
        
        try:
            with self.engine.begin() as conn:
                stmt = delete(self.table).where(self.table.c.user_id == user_id)
                conn.execute(stmt)
        except Exception as e:
            raise StorageError(f"Failed to clear keys: {e}")

## Hybrid Storage

In [ ]:
#| export
class HybridStorage:
    """
    Hybrid storage using both session and database.
    Session acts as a cache, database provides persistence.
    """
    
    def __init__(
        self,
        config: BYOKConfig,  # BYOK configuration object
        db_url: Optional[str] = None  # Optional database URL for persistent storage
    ):
        """Initialize hybrid storage with session and optional database backends"""
        self.config = config
        self.session_storage = SessionStorage(config)
        self.db_storage = DatabaseStorage(config, db_url) if db_url else None
    
    def store(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        key: APIKey  # API key object to store
    ) -> None:
        """Store in both session and database"""
        # Always store in session for fast access
        self.session_storage.store(request, key)
        
        # Store in database if available and user_id provided
        if self.db_storage and key.user_id:
            self.db_storage.store(request, key)
    
    def retrieve(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        provider: str,  # Provider name to retrieve key for
        user_id: Optional[str] = None  # User ID for database lookup
    ) -> Optional[APIKey]:  # API key object if found, None otherwise
        """Retrieve from session first, then database"""
        # Try session first (fast)
        key = self.session_storage.retrieve(request, provider)
        if key:
            return key
        
        # Try database if available
        if self.db_storage and user_id:
            key = self.db_storage.retrieve(request, provider, user_id)
            if key:
                # Cache in session for future requests
                self.session_storage.store(request, key)
            return key
        
        return None
    
    def delete(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        provider: str,  # Provider name to delete key for
        user_id: Optional[str] = None  # User ID for database deletion
    ) -> None:
        """Delete from both storages"""
        self.session_storage.delete(request, provider)
        if self.db_storage and user_id:
            self.db_storage.delete(request, provider, user_id)
    
    def list_providers(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        user_id: Optional[str] = None  # User ID for database lookup
    ) -> List[str]:  # Combined list of providers from both storages
        """List providers from both storages"""
        providers = set(self.session_storage.list_providers(request))
        
        if self.db_storage and user_id:
            providers.update(self.db_storage.list_providers(request, user_id))
        
        return list(providers)
    
    def clear_all(
        self,
        request: Any,  # FastHTML/Starlette request object with session
        user_id: Optional[str] = None  # User ID for database clearing
    ) -> None:
        """Clear from both storages"""
        self.session_storage.clear_all(request)
        if self.db_storage and user_id:
            self.db_storage.clear_all(request, user_id)

## Main Manager

In [ ]:
#| export
class BYOKManager:
    """
    Main manager for the BYOK system.
    Handles encryption and storage coordination.
    """
    
    def __init__(
        self,
        secret_key: str,  # Application secret key for encryption
        db_url: Optional[str] = None,  # Optional database URL for persistent storage (e.g., "sqlite:///keys.db")
        config: Optional[BYOKConfig] = None  # Optional configuration (uses defaults if not provided)
    ):
        """
        Initialize the BYOK manager.
        """
        self.config = config or BYOKConfig()
        
        # Setup encryption
        encryption_key = self.config.encryption_key or get_or_create_app_key(secret_key)
        self.encryptor = KeyEncryptor(encryption_key)
        
        # Setup storage based on configuration
        if db_url and self.config.storage_backend in (StorageBackend.DATABASE, StorageBackend.HYBRID):
            if self.config.storage_backend == StorageBackend.HYBRID:
                self.storage = HybridStorage(self.config, db_url)
            else:
                self.storage = DatabaseStorage(self.config, db_url)
        else:
            self.storage = SessionStorage(self.config)
    
    def set_key(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name (e.g., 'openai', 'anthropic')
        api_key: str,  # The API key to store
        user_id: Optional[str] = None,  # Optional user ID for database storage
        ttl: Optional[timedelta] = None  # Optional time-to-live for the key
    ) -> None:
        """
        Store an API key.
        """
        # Encrypt the key
        encrypted = self.encryptor.encrypt(api_key)
        
        # Calculate expiration
        expires_at = None
        if ttl or self.config.default_ttl:
            ttl = ttl or self.config.default_ttl
            expires_at = datetime.now() + ttl
        
        # Create key object
        key = APIKey(
            provider=provider,
            encrypted_value=encrypted,
            expires_at=expires_at,
            user_id=user_id
        )
        
        # Store it
        self.storage.store(request, key)
    
    def get_key(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name
        user_id: Optional[str] = None  # Optional user ID for database lookup
    ) -> Optional[str]:  # Decrypted API key or None if not found
        """
        Retrieve and decrypt an API key.
        """
        key = self.storage.retrieve(request, provider, user_id)
        if not key:
            return None
        
        try:
            return self.encryptor.decrypt(key.encrypted_value)
        except Exception:
            # If decryption fails, remove the corrupted key
            self.storage.delete(request, provider, user_id)
            return None
    
    def delete_key(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name
        user_id: Optional[str] = None  # Optional user ID
    ) -> None:
        """
        Delete an API key.
        """
        self.storage.delete(request, provider, user_id)
    
    def list_providers(
        self,
        request: Any,  # FastHTML/Starlette request object
        user_id: Optional[str] = None  # Optional user ID
    ) -> List[str]:  # List of provider names
        """
        List all providers with stored keys.
        """
        return self.storage.list_providers(request, user_id)
    
    def clear_keys(
        self,
        request: Any,  # FastHTML/Starlette request object
        user_id: Optional[str] = None  # Optional user ID
    ) -> None:
        """
        Clear all stored API keys.
        """
        self.storage.clear_all(request, user_id)
    
    def has_key(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name
        user_id: Optional[str] = None  # Optional user ID
    ) -> bool:  # True if key exists, False otherwise
        """
        Check if a key exists for a provider.
        """
        return self.get_key(request, provider, user_id) is not None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()